In [ ]:
!pwd

In [ ]:
cp ../mintpy_split_ion_upsample/inputs/ionStack.h5 ../mintpy_split_ion_upsample/inputs/ifgramStack.h5 .

In [ ]:
!modify_network.py ionStack.h5 --reset

In [ ]:
!modify_network.py ionStack.h5 --reference ifgramStack.h5

In [ ]:
!modify_network.py ifgramStack.h5 --reset

In [ ]:
!modify_network.py ifgramStack.h5  --reference ionStack.h5

In [ ]:
import h5py
import numpy as np

# 读取 ifgramStack.h5 文件
with h5py.File("ifgramStack.h5", "r+") as file:
    # 读取 "/dropIfgram" 数组和 "/unwrapPhase" 数组
    drop_ifgram = file["/dropIfgram"][:]
    unwrap_phase = file["/unwrapPhase"][:]
    date = file["/date"][:]
    bperp = file["/bperp"][:]

    # 确定要保留的数据的索引
    keep_indices = np.where(drop_ifgram == 1)[0]

    # 更新 "/unwrapPhase" 数组，只保留对应 dropIfgram 为 1 的数据
    unwrap_phase = unwrap_phase[keep_indices]
    date = date[keep_indices]
    drop_ifgram = drop_ifgram[keep_indices]
    bperp = bperp[keep_indices]

    # 删除对应 dropIfgram 为 0 的数据
    del file["/unwrapPhase"]
    file.create_dataset("/unwrapPhase", data=unwrap_phase)
    del file["/date"]
    file.create_dataset("/date", data=date)
    del file["/dropIfgram"]
    file.create_dataset("/dropIfgram", data=drop_ifgram)
    del file["/bperp"]
    file.create_dataset("/bperp", data=bperp)

    # 保存修改后的 ifgramStack.h5 文件
    file.flush()

# 读取 ionStack.h5 文件
with h5py.File("ionStack.h5", "r+") as file:
    # 读取 "/dropIfgram" 数组和 "/unwrapPhase" 数组
    drop_ifgram = file["/dropIfgram"][:]
    unwrap_phase = file["/unwrapPhase"][:]
    date = file["/date"][:]
    bperp = file["/bperp"][:]

    # 确定要保留的数据的索引
    keep_indices = np.where(drop_ifgram == 1)[0]

    # 更新 "/unwrapPhase" 数组，只保留对应 dropIfgram 为 1 的数据
    unwrap_phase = unwrap_phase[keep_indices]
    date = date[keep_indices]
    drop_ifgram = drop_ifgram[keep_indices]
    bperp = bperp[keep_indices]

    # 删除对应 dropIfgram 为 0 的数据
    del file["/unwrapPhase"]
    file.create_dataset("/unwrapPhase", data=unwrap_phase)
    del file["/date"]
    file.create_dataset("/date", data=date)
    del file["/dropIfgram"]
    file.create_dataset("/dropIfgram", data=drop_ifgram)
    del file["/bperp"]
    file.create_dataset("/bperp", data=bperp)

    # 保存修改后的 ionStack.h5 文件
    file.flush()

In [ ]:
!diff.py ifgramStack.h5 ionStack.h5

In [ ]:
ionStack_path = 'ionStack.h5'
with h5py.File(ionStack_path, 'r') as file:
    date = file['date']
    date = date[()]
ion_date_list = np.char.decode(date)
ion_date_list = [f'{start_date}_{end_date}' for start_date, end_date in ion_date_list]

In [ ]:
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
from mintpy.view import prep_slice, plot_slice


for i in range(len(ion_date_list)):
    print(i)
    opt = f' --wrap -c cmy --noverbose --zero-mask'
    cmd_list = [f'view.py ifgramStack.h5 unwrap*{ion_date_list[i]} {opt} --title wrap ',
                f'view.py ionStack.h5 *{ion_date_list[i]} {opt} --title ion --ref-yx 548 59', 
                f'view.py ifgramStack_diff_ionStack.h5  *{ion_date_list[i]} {opt} --title correct']

    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=[12, 8])
    for ax, cmd in zip(axs, cmd_list):
        data, atr, inps = prep_slice(cmd)
        plot_slice(ax, data, atr, inps)
    # 保存图片到pic_ion目录下，文件名为i
    plt.savefig(f'pic_ion/{ion_date_list[i]}.png')
    plt.show()
    plt.close()